# Filter Data Komentar YouTube

Notebook ini melakukan filtering data komentar untuk membuang:
1. **Noise** - spam, "wkwk", "hadir gan", dll
2. **Tidak Relevan** - komentar di luar konteks topik

**Output**: Data yang relevan dan bersih untuk preprocessing lebih lanjut

In [33]:
# Import libraries
import pandas as pd
import re
import json

# Load data
df = pd.read_csv('raw-scrape-yt.csv')
print(f"Total komentar: {len(df)}")
print("\nContoh data awal:")
df.head()

Total komentar: 2196

Contoh data awal:


,Video_ID,Teks_Komentar
0,MIo4tGN11j0,"berdoa tidak ada comment judol, amin."
1,MIo4tGN11j0,kereen.
2,MIo4tGN11j0,"Sempat mikir mau pindah ke negara sebelah, nge..."
3,MIo4tGN11j0,"Kalo kabur mau kemana ke Singapur ,emang di Si..."
4,MIo4tGN11j0,Jg ke malaysia.....pergi sono india bangla nep...


## 1. Filter Noise

Membuang komentar yang merupakan noise murni (spam, ketawa doang, terlalu pendek, dll)

In [34]:
def filter_noise(teks):
    """
    Return True jika komentar VALID (BUKAN noise).
    Return False jika komentar adalah NOISE (spam, pendek, dll).
    """
    if pd.isna(teks) or len(teks.strip()) < 3:
        return False
    
    teks_lower = teks.lower().strip()
    
    # Daftar pola noise yang harus dibuang
    noise_patterns = [
        r'^[@#]',  # Dimulai dengan @ atau #
        r'^\d+$',  # Hanya angka
        r'^[^\w\s]+$',  # Hanya simbol/emoji
        r'(subscribe|subs|subrek|gw klik subs)',  # Promosi subscribe
        r'(like|liek|liak|laik)\s*(dulu|dong|yuk)',  # Ajakan like
        r'^(wkwk|haha|xixi|kwkw|wkwkwk|hahaha)+$',  # Hanya ketawa
        r'^(amin|amiin|aamiin)+$',  # Hanya amin
        r'^(first)',  # Klaim urutan
        r'(giveaway|give away|kontes)',  # Promo
        r'(qna|q&a|q n a|q & a)',
        r'^(setuju|tidak setuju|ngga setuju)+$' #hanya setuju atau tidak setuju
    ]
    
    for pattern in noise_patterns:
        if re.search(pattern, teks_lower):
            return False
    
    return True

# --- Terapkan Filter Tahap 1 ---
print(f"Komentar sebelum filter: {len(df)}")

df_tahap_1 = df[df['Teks_Komentar'].apply(filter_noise)].copy()

print(f"Komentar setelah filter noise: {len(df_tahap_1)}")
print(f"Komentar noise terbuang: {len(df) - len(df_tahap_1)}")

Komentar sebelum filter: 2196
Komentar setelah filter noise: 2086
Komentar noise terbuang: 110


In [35]:
import pandas as pd
import re
from collections import Counter

# 1. Fungsi Tokenisasi Sederhana (Hanya ambil huruf)
def simple_tokenizer(text):
    # Hapus angka dan tanda baca, sisakan huruf
    text = re.sub(r'[^a-zA-Z\s]', ' ', str(text).lower())
    return text.split()

# 2. Kumpulkan Semua Kata
all_words = []
# Asumsi df_tahap_1 adalah dataframe kamu
for comment in df_tahap_1['Teks_Komentar']: # Ganti nama kolom jika beda
    all_words.extend(simple_tokenizer(comment))

# 3. Hitung Frekuensi
word_counts = Counter(all_words)

# 4. Filter Kata (Opsional tapi Recommended)
# Ambil kata yang muncul minimal 2 atau 3 kali. 
# (Kata yang cuma muncul 1x biasanya nama orang atau typo yang sangat random)
filtered_vocab = [word for word, count in word_counts.items() if count >= 2]

# 5. Sortir dari yang paling sering muncul
# Kita ubah jadi list string untuk dicopy
sorted_vocab = sorted(filtered_vocab, key=lambda x: word_counts[x], reverse=True)

# 6. Print hasilnya untuk di-copy ke AI
print(f"Total kata unik: {len(word_counts)}")
print(f"Kata yang akan dicek AI (muncul > 1x): {len(sorted_vocab)}")
print("="*30)

# Batching (Karena AI punya batas input, kita print per 500 kata)
batch_size = 500
for i in range(0, len(sorted_vocab), batch_size):
    batch = sorted_vocab[i:i+batch_size]
    print(f"\n--- BATCH {i//batch_size + 1} (Copy dari sini) ---")
    print(", ".join(batch))

Total kata unik: 8233
Kata yang akan dicek AI (muncul > 1x): 3463

--- BATCH 1 (Copy dari sini) ---
di, dan, yg, yang, indonesia, ini, itu, aja, orang, ada, kabur, ke, negara, bisa, bang, kita, dari, saya, untuk, dulu, tapi, jadi, nya, ga, tidak, mereka, kerja, mau, dengan, negeri, juga, banyak, gak, sama, ya, lebih, luar, apa, kalau, buat, gw, baik, kalo, lagi, sudah, bukan, karena, udah, rakyat, punya, semua, felix, aku, atau, pejabat, pemerintah, masih, banget, sendiri, akan, anak, adalah, sekarang, kalian, para, lu, indo, harus, dia, dalam, hidup, tahun, sangat, seperti, cuma, org, raymond, sih, baru, jangan, pada, semoga, balik, kan, kaya, lain, hanya, panji, allah, selalu, tau, salah, nasionalisme, nasionalis, gaji, satu, saja, sampai, mana, bikin, malah, pandji, saat, terus, gua, memang, cerdas, pak, mas, kenapa, hal, dgn, uang, pernah, cari, bilang, lah, tp, negri, pasti, kok, keluar, emang, gue, makin, bayar, masyarakat, pajak, tinggal, korupsi, jika, besar, bener, tanah, gk, 

## 2. Filter Konteks

Membuang komentar yang tidak relevan dengan topik (strategi LONGGAR: minimal ada 1 keyword)

In [36]:
kamus_dir = 'kamus_baku.json'
with open(kamus_dir, 'r', encoding='utf-8') as f:
    KAMUS_NORMALISASI = json.load(f)
    
def normalisasi_teks(teks):
    """
    Melakukan normalisasi slang words SEBELUM filtering.
    Contoh: "pjk mhal bgt" -> "pajak mahal banget"
    """
    if pd.isna(teks) or teks == '':
        return ''
    
    teks = str(teks).lower()
    # Hapus karakter aneh (opsional, tapi bagus untuk cleaning awal)
    # teks = re.sub(r'[^a-z0-9\s]', ' ', teks) 
    
    words = teks.split()
    
    # Ganti slang dengan kata baku
    cleaned_words = [KAMUS_NORMALISASI.get(w, w) for w in words]
    
    return ' '.join(cleaned_words)

In [37]:
df_tahap_1['text_normalized_temp'] = df_tahap_1['Teks_Komentar'].apply(normalisasi_teks)
print("Normalisasi teks selesai!")

Normalisasi teks selesai!


In [38]:
# 1. Migrasi & Negara Tujuan (Ingin Pindah)
keyword_migrasi = [
    'kabur', 'pindah', 'minggat', 'leave indo', 'luar negeri', 
    'paspor', 'visa', 'warga negara', 'wni', 'permanent resident',
    'singapura', 'singapur', 'singapore', 'australia', 'aussie', 
    'jepang', 'japan', 'eropa', 'jerman', 'belanda', 'indonesia', 'indo'
]

# 2. Ekonomi, Biaya Hidup & Pekerjaan (Paling Sensitif)
keyword_ekonomi = [
    'gaji', 'umr', 'biaya hidup', 'mahal', 'miskin', 'melarat',
    'sandwich', 'generasi sandwich', 'nyicil', 'cicilan', 'hutang',
    'pajak', 'ppn', 'tapera', 'bea cukai', 'pungli', 'palak',
    'loker', 'cari kerja', 'nganggur', 'pengangguran', 'phk',
    'boncos', 'ludes', 'cekek', 'cekik', 'naik terus'
]

# 3. Kritik Sistem, Pemerintah & Birokrasi
keyword_sistem = [
    'pemerintah', 'negara', 'birokrasi', 'politik', 'hukum', 
    'korupsi', 'kolusi', 'nepotisme', 'ordal', 'orang dalam',
    'tajam ke bawah', 'tumpul ke atas', 'blunder', 'omong kosong',
    'kiamat', 'bubar', 'hancur', 'gagal', 'suram', 'madesu'
]

# 4. Sarkasme & Sebutan Negara (+62)
keyword_sarkas = [
    'konoha', 'wakanda', 'negeri dongeng', 'negeri +62', '+62',
    'dagelan', 'badut', 'komedi', 'lawak', 'lucu'
]

# 5. Emosi Negatif Kuat (Marah/Capek/Putus Asa)
keyword_emosi = [
    'capek', 'lelah', 'muak', 'jijik', 'benci', 'kesal', 'sebel',
    'stres', 'depresi', 'gila', 'sakit', 'bingung', 'takut',
    'percuma', 'sia-sia', 'ga ada harapan', 'ga jelas', 'suram',
    'beban', 'susah', 'sulit', 'berat', 'mending',
    'sampah', 'tolol', 'bego', 'goblok' # Kata kasar sering kali indikator sentimen negatif kuat
]

# 6. Persetujuan (Validasi Keluhan)
keyword_setuju = [
    'setuju', 'bener banget', 'relate', 'valid', 'fakta', 'nyata'
]

# --- PENGGABUNGAN JADI SATU LIST UTUH ---
# Gunakan operator + untuk menggabungkan semua list di atas
FILTER_KEYWORDS = (
    keyword_migrasi + 
    keyword_ekonomi + 
    keyword_sistem + 
    keyword_sarkas + 
    keyword_emosi + 
    keyword_setuju
)

# Hapus duplikat (jika ada kata yang masuk di 2 kategori)
FILTER_KEYWORDS = list(set(FILTER_KEYWORDS))

print(f"Total keyword unik: {len(FILTER_KEYWORDS)}")

# --- FUNGSI FILTER ---
def filter_konteks(teks):
    if pd.isna(teks) or teks == '':
        return False
    teks_lower = str(teks).lower()
    return any(kata in teks_lower for kata in FILTER_KEYWORDS)

# Cara Pakai - Buat boolean mask lalu apply ke df_tahap_1 (DataFrame yang sama)
mask_relevan = df_tahap_1['text_normalized_temp'].apply(filter_konteks)
df_filtered = df_tahap_1[mask_relevan].copy()
print("="*40)
print("📊 HASIL FILTERING")
print("="*40)
print(f"Total keyword aktif: {len(FILTER_KEYWORDS)}")
print(f"Data Awal          : {len(df)}")
print(f"Data Relevan (Sisa): {len(df_filtered)}")
print(f"Data Terbuang      : {len(df) - len(df_filtered)}")
print(f"Persentase Lolos   : {(len(df_filtered)/len(df))*100:.2f}%")

Total keyword unik: 111
📊 HASIL FILTERING
Total keyword aktif: 111
Data Awal          : 2196
Data Relevan (Sisa): 1072
Data Terbuang      : 1124
Persentase Lolos   : 48.82%


## 3. Lihat Contoh Hasil Filter

In [39]:
# Tampilkan beberapa contoh komentar yang lolos filter
print("="*80)
print("CONTOH KOMENTAR YANG LOLOS FILTER")
print("="*80)

for i in range(min(10, len(df_filtered))):
    print(f"\n[{i+1}] {df_filtered['Teks_Komentar'].iloc[i][:100]}...")

print("\n" + "="*80)

CONTOH KOMENTAR YANG LOLOS FILTER

[1] Sempat mikir mau pindah ke negara sebelah, ngeapply citizenship. Tapi yah, aku cinta negara dan hara...

[2] Kalo kabur mau kemana ke Singapur ,emang di Singapur tinggal dimana rakyat Singapur aja nggak punya ...

[3] Klo sudah gelap susah terangnya lebih baik bubar...

[4] Siap siap ente motivator Indonesia gelap...dapet cipratan Marcela Santoso atau engga...

[5] Diskusi yg segar,menarik dan bermutu... Antara Raymond dan FS saling mengisi tanpa dominan atau meno...

[6] Jangan di take down ya video ini 😂 biar jadi pengingat agar tidak mudah terinfluence netizen buzzer ...

[7] Nah loh....marcela santoso koruptor CPO seorang pemodal Indonesia gelap. 😂😂😂😂...

[8] Pengen kabur tapi gak punya uang😭...

[9] Kalau mau pindah ke luar negeri ya silahkan saja...kenapa mesti koar koar ya. Kagak ngatri ane......

[10] 18.35_sosoknya mr. TM di era awal indo berdiri. Tiap masa_ada orang nya Tiap orang_ada masa nya Tuha...



In [40]:
# 1. Komentar yang dianggap noise
df_noise = df[~df.index.isin(df_tahap_1.index)]
print("="*80)
print("CONTOH KOMENTAR NOISE YANG DIBUANG")
print("="*80)
for i in range(min(5, len(df_noise))):
    print(f"[{i+1}] {df_noise['Teks_Komentar'].iloc[i]}")

# 2. Komentar yang dianggap tidak relevan (lolos noise, tapi tidak lolos konteks)
df_off_topic = df_tahap_1[~df_tahap_1.index.isin(df_filtered.index)]
print("\n" + "="*80)
print("CONTOH KOMENTAR TIDAK RELEVAN YANG DIBUANG")
print("="*80)
for i in range(min(5, len(df_off_topic))):
    print(f"[{i+1}] {df_off_topic['Teks_Komentar'].iloc[i]}")
print("\n" + "="*80)

CONTOH KOMENTAR NOISE YANG DIBUANG
[1] #kokori....
[2] #indonesiasakitt
[3] 🎉🎉🎉😢😢😢😢😢
[4] 🎉🎉🎉
[5] #kaburduluaja ganti dengan #hijrahduluaja,

CONTOH KOMENTAR TIDAK RELEVAN YANG DIBUANG
[1] berdoa tidak ada comment judol, amin.
[2] kereen.
[3] Jg ke malaysia.....pergi sono india bangla nepal serumpun....rupiah rupie
[4] gue mulai demen nih org
[5] Di sana tempat lahir beta , di buai di besarkan bunda , tempat berlindung di hari tua



## 4. Simpan Hasil Filter

Menyimpan data yang sudah difilter untuk digunakan di preprocessing

In [41]:
# Simpan hasil filter
output_file = 'data_filtered.csv'
df_filtered.to_csv(output_file, index=False, encoding='utf-8')

print("="*80)
print("✅ FILTER SELESAI!")
print("="*80)
print(f"\n📊 RINGKASAN:")
print(f"   Total komentar awal          : {len(df):,}")
print(f"   Setelah filter noise         : {len(df_tahap_1):,}")
print(f"   Setelah filter konteks       : {len(df_filtered):,}")
print(f"   Total terbuang               : {len(df) - len(df_filtered):,}")
print(f"   Persentase data valid        : {(len(df_filtered) / len(df) * 100):.2f}%")
print(f"\n💾 Data tersimpan di: {output_file}")
print(f"\n📝 Data ini siap untuk preprocessing di notebook 'preprocessing.ipynb'")
print("="*80)

✅ FILTER SELESAI!

📊 RINGKASAN:
   Total komentar awal          : 2,196
   Setelah filter noise         : 2,086
   Setelah filter konteks       : 1,072
   Total terbuang               : 1,124
   Persentase data valid        : 48.82%

💾 Data tersimpan di: data_filtered.csv

📝 Data ini siap untuk preprocessing di notebook 'preprocessing.ipynb'
